## Final Assignment
## Group C

## census_income dataset load from Google Bigquery into pandas df into csv file
no any processing (field types, missed values, outliers)


In [4]:
import pandas as pd

In [5]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r'C:\Users\sergiik\JupPython\keys\dev-republik-679bc5d8e22c.json'

In [6]:
%load_ext google.cloud.bigquery

In [7]:
from google.cloud import bigquery
client = bigquery.Client()

In [8]:
sql = """
    select 
      pd.*, 
      ri.relationship, 
      mi.functional_weight, mi.education, mi.education_num, 
      wi.workclass, wi.occupation, wi.capital_gain, wi.capital_loss, wi.hours_per_week, 
      ci.native_country, cl.country_name,
      tgt.income_bracket
    from 
      census_income.personal_data pd, 
      census_income.relationship_info ri, 
      census_income.misc_info mi,
      census_income.work_info wi, 
      census_income.country_info ci,
      census_income.country_labels cl,
      census_income.income_data tgt
    where 
      pd.relationship_label = ri.id
      and pd.id = mi.id
      and pd.id = wi.id
      and pd.id = ci.id
      and ci.native_country = cl.country_id
    and pd.id = tgt.id
    """
df = client.query(sql).to_dataframe()
df.head()

,id,age,sex,race,marital_status,relationship_label,relationship,functional_weight,education,education_num,workclass,occupation,capital_gain,capital_loss,hours_per_week,native_country,country_name,income_bracket
0,1623,17,Male,White,Married-civ-spouse,0,Husband,221129,9th,5,Private,Other-service,0,0,40,39,United-States,<=50K
1,14761,17,Male,White,Never-married,3,Own-child,191260,9th,5,Private,Other-service,1055,0,24,39,United-States,<=50K
2,14762,17,Male,Black,Never-married,3,Own-child,32124,9th,5,Local-gov,Other-service,0,0,9,39,United-States,<=50K
3,14769,17,Male,Black,Never-married,3,Own-child,127003,9th,5,?,?,0,0,40,39,United-States,<=50K
4,14796,17,Male,Black,Never-married,3,Own-child,133449,9th,5,Private,Other-service,0,0,26,39,United-States,<=50K


In [9]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  32561 non-null  int64 
 1   age                 32561 non-null  int64 
 2   sex                 32561 non-null  object
 3   race                32561 non-null  object
 4   marital_status      32561 non-null  object
 5   relationship_label  32561 non-null  int64 
 6   relationship        32561 non-null  object
 7   functional_weight   32561 non-null  int64 
 8   education           32561 non-null  object
 9   education_num       32561 non-null  int64 
 10  workclass           32561 non-null  object
 11  occupation          32561 non-null  object
 12  capital_gain        32561 non-null  int64 
 13  capital_loss        32561 non-null  int64 
 14  hours_per_week      32561 non-null  int64 
 15  native_country      32561 non-null  int64 
 16  country_name        32

,id,age,relationship_label,functional_weight,education_num,capital_gain,capital_loss,hours_per_week,native_country
count,32561.000000,32561.000000,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,16280.000000,38.581647,1.446362,1.897784e+05,10.080679,1077.648844,87.303830,40.437456,36.718866
std,9399.695394,13.640433,1.606771,1.055500e+05,2.572720,7385.292085,402.960219,12.347429,7.823782
min,0.000000,17.000000,0.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,8140.000000,28.000000,0.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000,39.000000
50%,16280.000000,37.000000,1.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000,39.000000
75%,24420.000000,48.000000,3.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000,39.000000
max,32560.000000,90.000000,5.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,41.000000


In [10]:
df.to_csv('census_income.csv')   # 4.5M of data

In [23]:
# percent of missing data per column and column's unique values
def df_info(df, showdf=False): 
    percent_missing = df.isnull().sum() / df.shape[0] * 100
    dfm = pd.DataFrame(df.columns, columns=['col_name'])
    dfm['%_missed'] = percent_missing.values
    dfm['n_missed'] = df.shape[0] - df.count().values
    dfm['value_counts'] = df.count().values
    dfm['sum'] = [ df[c].sum() if df[c].dtype != 'object' else '' for c in df.columns ]
    dfm['col_type'] = df.dtypes.values
    dfm['uniqcount'] = [ len(df[c].unique()) for c in df.columns ]
    dfm['unique_values_list'] = [df[c].unique() if str(df[c].dtype)=='object' else f'{df[c].min()}...{df[c].max()}'  for c in df.columns ]
    if showdf : 
        print(dfm)
    return dfm


In [24]:
df_info(df)

,col_name,%_missed,n_missed,value_counts,sum,col_type,uniqcount,unique_values_list
0,id,0.0,0,32561,530093080,int64,32561,0...32560
1,age,0.0,0,32561,1256257,int64,73,17...90
2,sex,0.0,0,32561,,object,2,"[ Male, Female]"
3,race,0.0,0,32561,,object,5,"[ White, Black, Other, Amer-Indian-Eskimo, ..."
4,marital_status,0.0,0,32561,,object,7,"[ Married-civ-spouse, Never-married, Widowed..."
5,relationship_label,0.0,0,32561,47095,int64,6,0...5
6,relationship,0.0,0,32561,,object,6,"[ Husband, Own-child, Unmarried, Not-in-fam..."
7,functional_weight,0.0,0,32561,6179373392,int64,21648,12285...1484705
8,education,0.0,0,32561,,object,16,"[ 9th, 10th, 11th, 12th, 7th-8th, HS-grad..."
9,education_num,0.0,0,32561,328237,int64,16,1...16
